In [1]:
%matplotlib inline
import pandas as pd
import csv as csv
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn import datasets, linear_model
import matplotlib
import matplotlib.pylab as pl
from matplotlib import rcParams
import sqlite3

:0: FutureWarning: IPython widgets are experimental and may change in the future.


In [2]:
ds = pd.read_csv('../raw_data/trip_data_1.csv', header = 0)

In [3]:
ds.head()

medallion                      hack_license  \
0  89D227B655E5C82AECF13C3F540D4CF4  BA96DE419E711691B9445D6A6307C170   
1  0BD7C8F5BA12B88E0B67BED28BEA73D8  9FD8F69F0804BDB5549F40E9DA1BE472   
2  0BD7C8F5BA12B88E0B67BED28BEA73D8  9FD8F69F0804BDB5549F40E9DA1BE472   
3  DFD2202EE08F7A8DC9A57B02ACB81FE2  51EE87E3205C985EF8431D850C786310   
4  DFD2202EE08F7A8DC9A57B02ACB81FE2  51EE87E3205C985EF8431D850C786310   

  vendor_id  rate_code store_and_fwd_flag      pickup_datetime  \
0       CMT          1                  N  2013-01-01 15:11:48   
1       CMT          1                  N  2013-01-06 00:18:35   
2       CMT          1                  N  2013-01-05 18:49:41   
3       CMT          1                  N  2013-01-07 23:54:15   
4       CMT          1                  N  2013-01-07 23:25:03   

      dropoff_datetime  passenger_count  trip_time_in_secs  trip_distance  \
0  2013-01-01 15:18:10                4                382            1.0   
1  2013-01-06 00:22:54                1                259            1.5   
2  2013-01-05 18:54:23                1                282            1.1   
3  2013-01-07 23:58:20                2                244            0.7   
4  2013-01-07 23:34:24                1                560            2.1   

   pickup_longitude  pickup_latitude  dropoff_longitude  dropoff_latitude  
0        -73.978165        40.757977         -73.989838         40.751171  
1        -74.006683        40.731781         -73.994499         40.750660  
2        -74.004707        40.737770         -74.009834         40.726002  
3        -73.974602        40.759945         -73.984734         40.759388  
4        -73.976250        40.748528         -74.002586         40.747868

In [4]:
len(ds.medallion)

14776615

In [5]:
fr = pd.read_csv('../raw_data/trip_fare_1.csv', header = 0)

In [6]:
fr.head()

medallion                      hack_license  \
0  89D227B655E5C82AECF13C3F540D4CF4  BA96DE419E711691B9445D6A6307C170   
1  0BD7C8F5BA12B88E0B67BED28BEA73D8  9FD8F69F0804BDB5549F40E9DA1BE472   
2  0BD7C8F5BA12B88E0B67BED28BEA73D8  9FD8F69F0804BDB5549F40E9DA1BE472   
3  DFD2202EE08F7A8DC9A57B02ACB81FE2  51EE87E3205C985EF8431D850C786310   
4  DFD2202EE08F7A8DC9A57B02ACB81FE2  51EE87E3205C985EF8431D850C786310   

   vendor_id      pickup_datetime  payment_type   fare_amount   surcharge  \
0        CMT  2013-01-01 15:11:48           CSH           6.5         0.0   
1        CMT  2013-01-06 00:18:35           CSH           6.0         0.5   
2        CMT  2013-01-05 18:49:41           CSH           5.5         1.0   
3        CMT  2013-01-07 23:54:15           CSH           5.0         0.5   
4        CMT  2013-01-07 23:25:03           CSH           9.5         0.5   

    mta_tax   tip_amount   tolls_amount   total_amount  
0       0.5            0              0            7.0  
1       0.5            0              0            7.0  
2       0.5            0              0            7.0  
3       0.5            0              0            6.0  
4       0.5            0              0           10.5

In [7]:
fr.columns = fr.columns.str.strip()

In [9]:
fr.columns.values

array(['medallion', 'hack_license', 'vendor_id', 'pickup_datetime',
       'payment_type', 'fare_amount', 'surcharge', 'mta_tax', 'tip_amount',
       'tolls_amount', 'total_amount'], dtype=object)

In [15]:
ds.trip_distance.describe()

count    14776615.000000
mean            2.770976
std             3.305923
min             0.000000
25%             1.000000
50%             1.700000
75%             3.060000
max           100.000000
Name: trip_distance, dtype: float64

In [18]:
len(fr[(fr.tip_amount!=0)&(fr.payment_type=='CSH')])

858

In [38]:
#Adjust for tips in cash payments
y_tip = fr.loc[(fr['payment_type']!='CSH'),'tip_amount'].as_matrix()
y_tip.shape = y_tip.shape + (1,)
x_fare = fr.loc[(fr['payment_type']!='CSH'),'fare_amount'].as_matrix()
x_fare.shape = x_fare.shape + (1,)
tip_regr = linear_model.LinearRegression()
tip_regr.fit(x_fare,y_tip)
cash_payments = fr.ix[fr['payment_type']=='CSH','fare_amount'].as_matrix()
cash_payments.shape = cash_payments.shape + (1,)
tips_predicted = tip_regr.predict(cash_payments)
fr.ix[fr['payment_type']=='CSH','tip_amount'] = tips_predicted
fr.ix[fr['payment_type']=='CSH','total_amount'] = (fr.ix[fr['payment_type']=='CSH','total_amount'] + tips_predicted.reshape(-1))

In [30]:
fr.payment_type.unique()

array(['CSH', 'DIS', 'NOC', 'CRD', 'UNK'], dtype=object)

In [40]:
fr.head()

medallion                      hack_license  \
0  89D227B655E5C82AECF13C3F540D4CF4  BA96DE419E711691B9445D6A6307C170   
1  0BD7C8F5BA12B88E0B67BED28BEA73D8  9FD8F69F0804BDB5549F40E9DA1BE472   
2  0BD7C8F5BA12B88E0B67BED28BEA73D8  9FD8F69F0804BDB5549F40E9DA1BE472   
3  DFD2202EE08F7A8DC9A57B02ACB81FE2  51EE87E3205C985EF8431D850C786310   
4  DFD2202EE08F7A8DC9A57B02ACB81FE2  51EE87E3205C985EF8431D850C786310   

  vendor_id      pickup_datetime payment_type  fare_amount  surcharge  \
0       CMT  2013-01-01 15:11:48          CSH          6.5        0.0   
1       CMT  2013-01-06 00:18:35          CSH          6.0        0.5   
2       CMT  2013-01-05 18:49:41          CSH          5.5        1.0   
3       CMT  2013-01-07 23:54:15          CSH          5.0        0.5   
4       CMT  2013-01-07 23:25:03          CSH          9.5        0.5   

   mta_tax  tip_amount  tolls_amount  total_amount  
0      0.5    1.409691             0      8.409691  
1      0.5    1.327724             0      8.327724  
2      0.5    1.245758             0      8.245758  
3      0.5    1.163791             0      7.163791  
4      0.5    1.901491             0     12.401491

In [37]:
fr.total_amount.describe()

count    14776615.000000
mean           13.952985
std            11.464686
min             2.500000
25%             7.700000
50%            10.500000
75%            15.500000
max           650.000000
Name: total_amount, dtype: float64

In [41]:
ds['avg_speed'] = ds['trip_distance'] / (ds['trip_time_in_secs'] / 3600)

In [46]:
ds = ds.drop('avg_speed',1)

In [48]:
len(ds[ds.trip_time_in_secs<=0])

34185

In [51]:
type(ds.pickup_datetime[1])

str

In [52]:
tot = pd.concat([ds,fr],axis=1, join='inner')

In [54]:
tot.head()

medallion                      hack_license  \
0  89D227B655E5C82AECF13C3F540D4CF4  BA96DE419E711691B9445D6A6307C170   
1  0BD7C8F5BA12B88E0B67BED28BEA73D8  9FD8F69F0804BDB5549F40E9DA1BE472   
2  0BD7C8F5BA12B88E0B67BED28BEA73D8  9FD8F69F0804BDB5549F40E9DA1BE472   
3  DFD2202EE08F7A8DC9A57B02ACB81FE2  51EE87E3205C985EF8431D850C786310   
4  DFD2202EE08F7A8DC9A57B02ACB81FE2  51EE87E3205C985EF8431D850C786310   

   vendor_id  rate_code store_and_fwd_flag      pickup_datetime  \
0        CMT          1                  N  2013-01-01 15:11:48   
1        CMT          1                  N  2013-01-06 00:18:35   
2        CMT          1                  N  2013-01-05 18:49:41   
3        CMT          1                  N  2013-01-07 23:54:15   
4        CMT          1                  N  2013-01-07 23:25:03   

      dropoff_datetime  passenger_count  trip_time_in_secs  trip_distance  \
0  2013-01-01 15:18:10                4                382            1.0   
1  2013-01-06 00:22:54                1                259            1.5   
2  2013-01-05 18:54:23                1                282            1.1   
3  2013-01-07 23:58:20                2                244            0.7   
4  2013-01-07 23:34:24                1                560            2.1   

       ...                           hack_license  vendor_id  \
0      ...       BA96DE419E711691B9445D6A6307C170        CMT   
1      ...       9FD8F69F0804BDB5549F40E9DA1BE472        CMT   
2      ...       9FD8F69F0804BDB5549F40E9DA1BE472        CMT   
3      ...       51EE87E3205C985EF8431D850C786310        CMT   
4      ...       51EE87E3205C985EF8431D850C786310        CMT   

       pickup_datetime  payment_type fare_amount surcharge mta_tax tip_amount  \
0  2013-01-01 15:11:48           CSH         6.5       0.0     0.5   1.409691   
1  2013-01-06 00:18:35           CSH         6.0       0.5     0.5   1.327724   
2  2013-01-05 18:49:41           CSH         5.5       1.0     0.5   1.245758   
3  2013-01-07 23:54:15           CSH         5.0       0.5     0.5   1.163791   
4  2013-01-07 23:25:03           CSH         9.5       0.5     0.5   1.901491   

  tolls_amount  total_amount  
0            0      8.409691  
1            0      8.327724  
2            0      8.245758  
3            0      7.163791  
4            0     12.401491  

[5 rows x 25 columns]

In [55]:
len(tot.medallion)

14776615

In [56]:
tot.to_csv('../csv/jan2013_all')